In [9]:
from sklearn.dummy import DummyClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier

In [8]:
pip install catboost


                                              0.0/101.7 MB ? eta -:--:--
                                              0.0/101.7 MB 2.0 MB/s eta 0:00:50
                                              0.1/101.7 MB 1.5 MB/s eta 0:01:07
                                              0.2/101.7 MB 2.1 MB/s eta 0:00:50
                                              0.3/101.7 MB 2.3 MB/s eta 0:00:44
                                              0.5/101.7 MB 2.6 MB/s eta 0:00:40
                                              0.7/101.7 MB 2.6 MB/s eta 0:00:39
                                              0.9/101.7 MB 3.0 MB/s eta 0:00:34
                                              1.2/101.7 MB 3.4 MB/s eta 0:00:30
                                              1.2/101.7 MB 3.3 MB/s eta 0:00:31
                                              1.3/101.7 MB 3.2 MB/s eta 0:00:32
                                              1.3/101.7 MB 3.2 MB/s eta 0:00:32
                                              


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
train = pd.read_csv("train.tsv", sep='\t')
test = pd.read_csv("test.tsv", sep='\t')

In [11]:
trainX = train.drop(columns=["y", "id"])
trainY = train["y"]
testX = test.drop(columns=["id"])
submission = test[["id"]].copy()

In [20]:
train.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x992,x993,x994,x995,x996,x997,x998,x999,x1000,y
0,1,80,?,-10,-20,-10,150,?,-510,?,...,20,60,-10,50,?,-50,?,-3860,-270,P
1,2,70,20,-20,?,-10,?,360,-440,-400,...,20,?,-90,50,?,-60,0,-3250,-750,P
2,3,80,?,20,-10,10,?,190,-500,880,...,10,?,?,60,-1310,-40,0,-3410,-630,N
3,4,60,?,?,-40,?,?,580,?,-200,...,20,60,40,60,?,-50,?,-3020,?,P
4,5,70,?,?,-10,0,230,?,30,390,...,20,?,-60,?,-1410,-50,-40,-3500,-420,P


In [28]:
trainX = trainX.apply(pd.to_numeric, errors='coerce')

# Заполнение пропусков медианными значениями
trainX.fillna(trainX.median(), inplace=True)

In [29]:
trainX.dtypes

x1       float64
x2       float64
x3       float64
x4       float64
x5       float64
          ...   
x996     float64
x997     float64
x998     float64
x999     float64
x1000    float64
Length: 1000, dtype: object

In [35]:
trainX.head(10)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x991,x992,x993,x994,x995,x996,x997,x998,x999,x1000
0,80.0,60.0,-10.0,-20.0,-10.0,150.0,350.0,-510.0,60.0,-10.0,...,50.0,20.0,60.0,-10.0,50.0,-1240.0,-50.0,-20.0,-3860.0,-270.0
1,70.0,20.0,-20.0,-10.0,-10.0,140.0,360.0,-440.0,-400.0,0.0,...,50.0,20.0,40.0,-90.0,50.0,-1240.0,-60.0,0.0,-3250.0,-750.0
2,80.0,60.0,20.0,-10.0,10.0,140.0,190.0,-500.0,880.0,0.0,...,50.0,10.0,40.0,20.0,60.0,-1310.0,-40.0,0.0,-3410.0,-630.0
3,60.0,60.0,10.0,-40.0,0.0,140.0,580.0,-450.0,-200.0,-10.0,...,50.0,20.0,60.0,40.0,60.0,-1240.0,-50.0,-20.0,-3020.0,-670.0
4,70.0,60.0,10.0,-10.0,0.0,230.0,350.0,30.0,390.0,0.0,...,50.0,20.0,40.0,-60.0,50.0,-1410.0,-50.0,-40.0,-3500.0,-420.0
5,70.0,60.0,40.0,-60.0,0.0,190.0,280.0,-650.0,540.0,-10.0,...,60.0,10.0,40.0,130.0,30.0,-1230.0,-50.0,-20.0,-3250.0,-670.0
6,60.0,10.0,10.0,-20.0,20.0,140.0,440.0,-40.0,60.0,0.0,...,50.0,20.0,60.0,90.0,60.0,-1240.0,-50.0,-20.0,-3250.0,-670.0
7,70.0,80.0,-10.0,-10.0,10.0,-30.0,350.0,-290.0,60.0,-10.0,...,60.0,20.0,50.0,20.0,60.0,-1140.0,-50.0,0.0,-3250.0,-700.0
8,80.0,100.0,-20.0,-30.0,-10.0,140.0,330.0,-450.0,790.0,0.0,...,50.0,20.0,10.0,120.0,50.0,-1210.0,-50.0,-20.0,-4050.0,-420.0
9,80.0,110.0,-20.0,-60.0,-20.0,160.0,320.0,-210.0,-490.0,-10.0,...,50.0,20.0,40.0,30.0,30.0,-1240.0,-60.0,0.0,-2120.0,-710.0


In [39]:
testX = testX.apply(pd.to_numeric, errors='coerce')

# Заполнение пропусков медианными значениями
testX.fillna(testX.median(), inplace=True)
testX.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x991,x992,x993,x994,x995,x996,x997,x998,x999,x1000
0,70.0,90.0,-100.0,-10.0,-10.0,140.0,260.0,-370.0,-280.0,0.0,...,50.0,20.0,10.0,140.0,50.0,-1230.0,-60.0,-20.0,-3200.0,-950.0
1,60.0,40.0,-40.0,-20.0,-10.0,110.0,330.0,-450.0,60.0,0.0,...,50.0,20.0,40.0,20.0,50.0,-1220.0,-50.0,-20.0,-3560.0,-920.0
2,80.0,60.0,10.0,-60.0,0.0,140.0,360.0,-540.0,60.0,0.0,...,50.0,20.0,40.0,100.0,50.0,-1170.0,-50.0,-10.0,-3240.0,-680.0
3,80.0,90.0,40.0,10.0,0.0,190.0,490.0,-380.0,60.0,-10.0,...,50.0,20.0,20.0,-40.0,40.0,-1230.0,-50.0,-40.0,-3240.0,-680.0
4,80.0,70.0,40.0,-10.0,0.0,70.0,470.0,-340.0,630.0,0.0,...,50.0,10.0,30.0,20.0,40.0,-1700.0,-60.0,-20.0,-3240.0,-420.0


In [40]:
ct_model = CatBoostClassifier(random_state=42)
ct_model.fit(
    X=trainX, 
    y=trainY, 
)
test_pred = ct_model.predict(testX)

Learning rate set to 0.037023
0:	learn: 0.6647034	total: 49.8ms	remaining: 49.7s
1:	learn: 0.6383240	total: 91.7ms	remaining: 45.8s
2:	learn: 0.6135716	total: 134ms	remaining: 44.5s
3:	learn: 0.5899433	total: 178ms	remaining: 44.2s
4:	learn: 0.5680692	total: 222ms	remaining: 44.1s
5:	learn: 0.5499343	total: 273ms	remaining: 45.2s
6:	learn: 0.5309561	total: 316ms	remaining: 44.8s
7:	learn: 0.5132630	total: 359ms	remaining: 44.5s
8:	learn: 0.4970162	total: 401ms	remaining: 44.2s
9:	learn: 0.4827743	total: 450ms	remaining: 44.5s
10:	learn: 0.4698237	total: 502ms	remaining: 45.1s
11:	learn: 0.4579561	total: 544ms	remaining: 44.8s
12:	learn: 0.4468241	total: 583ms	remaining: 44.2s
13:	learn: 0.4364967	total: 622ms	remaining: 43.8s
14:	learn: 0.4269737	total: 661ms	remaining: 43.4s
15:	learn: 0.4186123	total: 711ms	remaining: 43.7s
16:	learn: 0.4102698	total: 749ms	remaining: 43.3s
17:	learn: 0.4023854	total: 787ms	remaining: 42.9s
18:	learn: 0.3946723	total: 829ms	remaining: 42.8s
19:	learn

In [41]:
submission["y"] = test_pred

In [43]:
submission.to_csv("submission.tsv", sep='\t')